In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from dfply import *
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [3]:
wfrcboundarypath = r"data/WFRCBoundary2018/WFRCBoundary2018.shp"
utahblockspath = r"data/Utah_Census_Block_Groups_2020/CensusBlockGroups2020.shp"
minoritycsv = r"data/ACSDT5Y2020.B03002/ACSDT5Y2020.B03002_data_with_overlays_2022-06-07T113506.csv"
vehiclescsv = r"data/ACSDT5Y2020.B25044/ACSDT5Y2020.B25044_data_with_overlays_2022-06-07T123124.csv"
incomecsv = r"data/ACSDT5Y2020.C17002/ACSDT5Y2020.C17002_data_with_overlays_2022-06-07T123423.csv"

In [5]:
wfrcboundary = pd.DataFrame.spatial.from_featureclass(wfrcboundarypath)
utahblocks = pd.DataFrame.spatial.from_featureclass(utahblockspath)
minority = pd.read_csv(minoritycsv)
vehicles = pd.read_csv(vehiclescsv)
income = pd.read_csv(incomecsv)

In [48]:
MinorityTable = (minority >>
    tail(minority.shape[0] - 1) >>
    rename(Population=X.B03002_001E,White_E=X.B03002_003E,White_M=X.B03002_003M) >>
    mutate(Population = (X.Population).astype(float),
           White_E = (X.White_E).astype(float),
           White_M = (X.White_M).astype(float)
          ) >>
    mutate(Minority = X.Population - X.White_E) >>
    mutate(PercMinori = (X.Minority/X.Population).round(3)) >>
    select("NAME","GEO_ID","Population","Minority","PercMinori") >>
    mutate(Perc_Minorit = if_else(X.PercMinori > 0.4,1,0))
)

In [ ]:
VehicleTable = (vehicles >>
    tail(minority.shape[0] - 1) >>
    rename(TotalHH=X.B25044_001E,OwnerZeroVehs_E=X.B25044_003E,OwnerZeroVehs_M=X.B25044_003M,RenterZeroVehs_E=X.B25044_010E,RenterZeroVehs_M=X.B25044_010M) >>
    mutate(TotalHH = (X.TotalHH).astype(float),
           OwnerZeroVehs_E = (X.OwnerZeroVehs_E).astype(float),
           OwnerZeroVehs_M = (X.OwnerZeroVehs_M).astype(float),
           RenterZeroVehs_E = (X.RenterZeroVehs_E).astype(float),
           RenterZeroVehs_M = (X.RenterZeroVehs_M).astype(float)
         ) >>
    mutate(ZeroCar = X.OwnerZeroVehs_E + X.RenterZeroVehs_E) >>
    mutate(PercZeroCa = (X.ZeroCar/X.TotalHH).round(3)) >>
    select("NAME","GEO_ID","ZeroCar","PercZeroCa") >>
    mutate(Perc_ZeroCar = if_else(X.PercZeroCa > 0.1,1,0))
)
VehicleTable

In [ ]:
#PovertyTable2020 <- income[-1,] %>%
#  rename("pop" = C17002_001E, "superpoor_e" = C17002_002E, "superpoor_m" = C17002_002M, "poor_e" = C17002_003E, "poor_m" = C17002_003M) %>%
#  mutate(pop = as.numeric(pop), superpoor_e = as.numeric(superpoor_e), superpoor_m = as.numeric(superpoor_m), poor_e = as.numeric(poor_e), poor_m = as.numeric(poor_m)) %>%
#  mutate(Poverty = superpoor_e + poor_e) %>%
#  mutate(PercPovert = round(Poverty/pop,3)) %>%
#  select(NAME,GEO_ID,Poverty,PercPovert) %>%
#  mutate(Perc_Pov = ifelse(PercPovert > 0.25,1,0))

PovertyTable = (income >>


)